In [ ]:
function generate_data(nr_samples::Int64)

    # sample weights
    w = rand(nr_samples,2)

    # sample appraisal
    y = zeros(Float64, nr_samples)
    for k = 1:nr_samples
        y[k] = 1.0*(w[k,1] > 0.5)*(w[k,2] < 0.5)
    end

    # return data
    return y, w

end;

In [ ]:
dat_to_col(data) = map(x -> x == 1 ? "red" : "blue", Int64.(round.(data)))
data_y, data_x = generate_data(200);
scatter(data_x[:,1], data_x[:,2], color=dat_to_col(data_y), grid=true, xlabel="g1", ylabel="g2", label=false)

In [ ]:
function f(params)
    fe = inference_flow_classifier(Float64.(data_y), [data_x[k,:] for k=1:size(data_x,1)], params)
    return fe
end;

In [ ]:
using Optim
res_train = optimize(f, randn(12), LBFGS(), Optim.Options(store_trace = true, show_trace = true, f_tol=1e-6), autodiff=:forward);

In [ ]:
using ImageCore
using LinearAlgebra

est_params = Optim.minimizer(res_train)
cmap = colorsigned(color("blue"), color("magenta"), color("red")) ∘ scalesigned(0, 0.5, 1.0)
colors = cmap.(data_y)
inferred_model = deepcopy(compile(model, est_params))
trans_data_x_1 = hcat(map((x) -> forward(inferred_model, x), [data_x[k,:] for k=1:size(data_x,1)])...)'
trans_data_x_2 = map((x) -> dot([1, 1], x), [trans_data_x_1[k,:] for k=1:size(data_x,1)])
trans_data_x_2_split = [trans_data_x_2[data_y .>= 0.5], trans_data_x_2[data_y .<= 0.5]]
plt1 = scatter(data_x[:,1], data_x[:,2], color=colors)
plt2 = scatter(trans_data_x_1[:,1], trans_data_x_1[:,2], color=colors)
hist = histogram(trans_data_x_2_split; stacked=true, normalize=:density, bins=60,  fillcolor=["red" "blue"], label = ["Positive" "Negative"], legend=:topleft)
plot(plt1, plt2, hist, size=(1600, 400), legend=false, layout=(1, 3))

In [ ]:
# test
function f_input(input)
    fe = inference_flow_classifier_input(input, model, est_params)
    return fe
end;

In [ ]:
est_params = Optim.minimizer(res_train)

In [ ]:
res = optimize(f_input, rand(2), LBFGS(), Optim.Options(iterations = 100, store_trace = true, show_trace = true), autodiff=:forward)

In [ ]:
optimum = Optim.minimizer(res)

In [ ]:
z = zeros(100,100)
for kx = 1:100
    for ky = 1:100
        z[kx, ky] = generate_user_response([kx/100, ky/100])
    end
end